# Experiments with GP pushing and then simulated annealing for finalization

In [1]:
%load_ext autoreload
%autoreload 2
from DLC_for_WBFM.utils.projects.finished_project_data import finished_project_data
import numpy as np
import matplotlib.pyplot as plt
import pickle
import napari, zarr

In [2]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
dat = finished_project_data.load_final_project_data_from_config(fname)

In [3]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\match_dat.pickle"
with open(fname, 'rb') as f:
    matches = pickle.load(f)
    
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\frame_dat.pickle"
with open(fname, 'rb') as f:
    frames = pickle.load(f)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Visualize the pushed neurons on the next frame

In [4]:
pair = (0, 1)
print(matches[pair])
print(frames[pair[0]], frames[pair[1]])

FramePair with 134 matches 

ReferenceFrame:
Frame index: 0 
Number of neurons: 148 
Number of keypoints: 148 
ReferenceFrame:
Frame index: 1 
Number of neurons: 151 
Number of keypoints: 151 



In [5]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [6]:
# Get gp pushed locations

gp_zxy = matches[pair].gp_pushed_locations
# Replace z coordinate, which is somehow crazy
gp_zxy[:, 0] = frames[pair[0]].neuron_locs[:, 0]
gp_zxy = gp_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
# Replace z coordinate, which is somehow crazy
# affine_zxy[:, 0] = frames[pair[0]].neuron_locs[:, 0]
affine_zxy = affine_zxy[:, [0,2,1]]

In [7]:
# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

gp_xyz_matched = [row for i, row in enumerate(gp_zxy) if i in has_match_ind]
gp_xyz_unmatched = [row for i, row in enumerate(gp_zxy) if i not in has_match_ind]

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [8]:
# # Plot using napari
# v = napari.view_image(dat.red_data[pair[1]], ndisplay=3)
# # v.add_points(gp_xyz_matched, size=3, face_color='green', n_dimensional=True)
# v.add_points(gp_xyz_unmatched, size=3, face_color='red', n_dimensional=True)
# # v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True)
# v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
# v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
# v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

In [9]:

# Get gradient images (not in z)
# vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)
# thresh = 1.5*np.mean(vol_filtered)
# mask = vol_filtered > thresh
# vol_gradient_x = sobel_h(vol_filtered, mask)
# vol_gradient_y = sobel_v(vol_filtered, mask)

# Actually do it (no annealing)

In [10]:
# from skimage.filters import gaussian, sobel_h, sobel_v
from scipy.spatial import KDTree

In [11]:
pair = (1, 2)

In [12]:
# Get initial affine locations
affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Get image
vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)

# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [13]:
# Only check the ones with no match
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}
has_match_ind = list(f0_to_f1_matches.keys())
no_match_ind = list(range(frames[pair[0]].num_neurons()))
[no_match_ind.remove(i) for i in has_match_ind]

# Set up kdtree to check nearest neighbor
tree = KDTree(target_zxy)

# Parameters
brightness_thresh = 25
distance_thresh = 5.0

In [14]:
new_f1_zxy = []
new_matches = []
next_f1_ind = frames[pair[1]].num_neurons() + 1

for ind_f0 in no_match_ind:
    pushed_zxy = affine_zxy[ind_f0, :]
    z, x, y = int(pushed_zxy[0]), int(pushed_zxy[1]), int(pushed_zxy[2])
    brightness = vol[z, x, y]
    
    nn_dist, _ = tree.query(pushed_zxy, k=1)
    
    to_keep = (brightness > brightness_thresh) and (nn_dist > distance_thresh)
    print(to_keep, brightness, nn_dist)
    if to_keep:
        new_matches.append([ind_f0, next_f1_ind, 1.0])
        next_f1_ind += 1
        new_f1_zxy.append(pushed_zxy)

print(f"Added {len(new_matches)} neurons")

False 0 278.634677184079
False 22 6.514810434866421
True 40 8.798306660844792
False 31 3.65074508888712
False 34 4.273267236144742
True 36 9.445009640142503
True 27 12.186874485455414
True 35 7.80057086110525
Added 4 neurons


In [15]:
# # Plot using napari
# v = napari.view_image(dat.red_data[pair[1]], ndisplay=3)
# v.add_points(new_f1_zxy, size=3, face_color='green', symbol='x', n_dimensional=True)
# v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
# v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
# v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

In [16]:
f0, f1 = frames[pair[0]], frames[pair[1]]

In [17]:
len(f0.keypoint_locs)

151

# Use piecewise affine with only matched neurons

In [18]:
from skimage.transform import PiecewiseAffineTransform, warp

In [19]:
pair = (0, 1)
print(matches[pair])
print(frames[pair[0]], frames[pair[1]])

FramePair with 134 matches 

ReferenceFrame:
Frame index: 0 
Number of neurons: 148 
Number of keypoints: 148 
ReferenceFrame:
Frame index: 1 
Number of neurons: 151 
Number of keypoints: 151 



In [20]:
m = matches[pair]
f0, f1 = frames[pair[0]], frames[pair[1]]

In [21]:
f0_to_f1 = m.get_f0_to_f1_dict()

In [22]:
src_matched = np.array([f0.keypoint_locs[k,[2,1]] for k in f0_to_f1.keys()])
src_all_xy = np.array(f0.keypoint_locs[:,[2,1]])
src_z = np.array(f0.keypoint_locs[:,[0]])

dst_matched = np.array([f1.keypoint_locs[v,[2,1]] for v in f0_to_f1.values()])
dst_all_xy = np.array(f0.keypoint_locs[:,[2,1]])
dst_z = np.array(f0.keypoint_locs[:,[0]])

In [23]:

tform = PiecewiseAffineTransform()
tform.estimate(src_matched, dst_matched)

True

In [24]:
transformed = tform.inverse(dst_all_xy)
# src_transformed = tform.inverse(src)

transformed = np.where(transformed==-1, np.nan, transformed)

transformed = np.hstack([dst_z, transformed]) #[:, [0,2,1]]

In [25]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [26]:
# # Plot using napari
# v = napari.view_image(dat.red_data[pair[0]], ndisplay=3, name='f0_red')
# v.add_image(dat.red_data[pair[1]], name='f1_red')
# v.add_points(transformed, size=3, face_color='red', n_dimensional=True, name='piecewise_affine_transformed')
# v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True, name='f0_affine_matched')
# # v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
# v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True, name='f1_matched')
# v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True, name='f1_unmatched')

In [27]:

# tform.estimate(dst, src)
# src_transformed = tform.inverse(src)
# # src_transformed = tform.inverse(f0.keypoint_locs[:,[2,1]])

# src_transformed[:3], dst[:3], src[:3]

# Piecewise affine, but add additional ORB features

In [28]:
from skimage.transform import PiecewiseAffineTransform, warp

In [29]:
pair = (0, 1)
pair_obj = matches[pair]
print(pair_obj)
f0, f1 = frames[pair[0]], frames[pair[1]]
print(f0, f1)

FramePair with 134 matches 

ReferenceFrame:
Frame index: 0 
Number of neurons: 148 
Number of keypoints: 148 
ReferenceFrame:
Frame index: 1 
Number of neurons: 151 
Number of keypoints: 151 



In [30]:
# Update Frame classes, if old style
pair_obj.frame0, pair_obj.frame1 = f0, f1

In [31]:
err

NameError: name 'err' is not defined

In [41]:
f0

ReferenceFrame with 148 neurons 

In [ ]:
video_dat = dat.red_data[pair[0],...]

# f0.detect_non_neuron_keypoints(video_dat, append_to_existing_keypoints=True)
f0.encode_all_keypoints(video_dat, z_depth=0)
# f0.build_nontrivial_keypoint_to_neuron_mapping(neuron_feature_radius=3)

In [ ]:
from DLC_for_WBFM.utils.feature_detection.utils_features import convert_to_grayscale
import cv2

loc = f0.neuron_locs[0]

z, x, y = loc
kp_2d = cv2.KeyPoint(x, y, 31.0)
base_2d_encoder = cv2.xfeatures2d.VGG_create()
im_3d_gray = [convert_to_grayscale(xy).astype('uint8') for xy in video_dat]
im_2d = im_3d_gray[int(5)]
_, this_ds = base_2d_encoder.compute(im_2d, [kp_2d])

In [ ]:
f0.all_features.shape

In [ ]:
m = matches[pair]
f0, f1 = frames[pair[0]], frames[pair[1]]

In [ ]:
f0_to_f1 = m.get_f0_to_f1_dict()

In [ ]:
src_matched = np.array([f0.neuron_locs[k,[2,1]] for k in f0_to_f1.keys()])
src_all_xy = np.array(f0.neuron_locs[:,[2,1]])
src_z = np.array(f0.neuron_locs[:,[0]])

dst_matched = np.array([f1.neuron_locs[v,[2,1]] for v in f0_to_f1.values()])
dst_all_xy = np.array(f0.neuron_locs[:,[2,1]])
dst_z = np.array(f0.neuron_locs[:,[0]])

In [ ]:
tform = PiecewiseAffineTransform()
tform.estimate(src_matched, dst_matched)

In [ ]:
transformed = tform.inverse(dst_all_xy)
# src_transformed = tform.inverse(src)

transformed = np.where(transformed==-1, np.nan, transformed)

transformed = np.hstack([dst_z, transformed]) #[:, [0,2,1]]

In [ ]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [ ]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[0]], ndisplay=3, name='f0_red')
v.add_image(dat.red_data[pair[1]], name='f1_red')
v.add_points(transformed, size=3, face_color='red', n_dimensional=True, name='piecewise_affine_transformed')
v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True, name='f0_affine_matched')
# v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True, name='f1_matched')
v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True, name='f1_unmatched')

# Basic encoding tests: do matches change based on VGG z depth?

In [50]:
from DLC_for_WBFM.utils.feature_detection.class_frame_pair import calc_FramePair_like

In [47]:
pair = (0, 1)
pair_obj = matches[pair]
print(pair_obj)
f0, f1 = frames[pair[0]], frames[pair[1]]
# print(f0, f1)

FramePair with 134/148 matches 



In [49]:
# Fix old style
pair_obj.frame0, pair_obj.frame1 = f0, f1

pair_obj.add_affine_to_candidates=True
pair_obj.add_gp_to_candidates=True

In [74]:
f0_video_dat = dat.red_data[pair[0],...]
f1_video_dat = dat.red_data[pair[1],...]

# Redo encoding with no z depth
f0.rebuild_keypoints()
f0.encode_all_keypoints(f0_video_dat, z_depth=5)
f1.rebuild_keypoints()
f1.encode_all_keypoints(f1_video_dat, z_depth=5)
# Note: do not need to change keypoint-neuron matching

# Redo matching (new pair object)
new_pair5 = calc_FramePair_like(pair_obj, f0, f1)

# Compare
print(new_pair5)

Overwriting existing keypoints...


  0%|          | 0/148 [00:00<?, ?it/s]

Overwriting existing keypoints...


  0%|          | 0/151 [00:00<?, ?it/s]

C:\Users\charles.fieseler\Anaconda3\envs\segmentation\lib\site-packages\sklearn\gaussian_process\kernels.py:409: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__sigma_0 is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
C:\Users\charles.fieseler\Anaconda3\envs\segmentation\lib\site-packages\sklearn\gaussian_process\kernels.py:409: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__sigma_0 is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
C:\Users\charles.fieseler\Anaconda3\envs\segmentation\lib\site-packages\sklearn\gaussian_process\kernels.py:409: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarni

FramePair with 134/148 matches 



In [85]:
f0_video_dat = dat.red_data[pair[0],...]
f1_video_dat = dat.red_data[pair[1],...]

# Redo encoding with no z depth
f0.rebuild_keypoints()
f0.encode_all_keypoints(f0_video_dat, z_depth=0)
f1.rebuild_keypoints()
f1.encode_all_keypoints(f1_video_dat, z_depth=0)

# Redo matching (new pair object)
new_pair0 = calc_FramePair_like(pair_obj, f0, f1)

# Compare
print(new_pair0)

Overwriting existing keypoints...


  0%|          | 0/148 [00:00<?, ?it/s]

Overwriting existing keypoints...


  0%|          | 0/151 [00:00<?, ?it/s]

FramePair with 134/148 matches 



C:\Users\charles.fieseler\Anaconda3\envs\segmentation\lib\site-packages\sklearn\gaussian_process\kernels.py:409: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__sigma_0 is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
C:\Users\charles.fieseler\Anaconda3\envs\segmentation\lib\site-packages\sklearn\gaussian_process\kernels.py:409: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarning)


In [104]:
f0_map_0 = new_pair0.get_f0_to_f1_dict()
f0_map_5 = new_pair5.get_f0_to_f1_dict()

f0_conf_0 = new_pair0.get_pair_to_conf_dict()
f0_conf_5 = new_pair5.get_pair_to_conf_dict()

In [105]:
num_mismatches = 0
all_keys = set(f0_map_0.keys())
all_keys.update(tuple(f0_map_5.keys()))
for i in all_keys:
    m0, m5 = f0_map_0.get(i,None), f0_map_5.get(i,None)
    if m0 != m5:
        print(i, ':', m0, m5)
        print('conf: ', f0_conf_0.get((i, m0), None), f0_conf_5.get((i, m5), None))
        num_mismatches += 1
print(f"Number of mismatches {num_mismatches}")

0 : 68 132
conf:  0.5802337759857685 0.19394127314555554
8 : 133 None
conf:  0.20329414440551336 None
53 : 28 3
conf:  0.034576344527244174 0.8329925214772761
65 : None 28
conf:  None 0.059030207580770044
68 : 110 87
conf:  0.5058319859765046 0.6010213441295417
89 : 72 141
conf:  0.5223284961253418 0.435534410306495
92 : 123 147
conf:  0.8802500567772639 0.882614979999581
96 : 26 None
conf:  0.5436642479928631 None
108 : 136 47
conf:  0.03495570191596903 0.04567313772047888
112 : 125 99
conf:  0.037409675256080334 0.04925594002648937
119 : None 72
conf:  None 0.27807912347449676
125 : 64 26
conf:  0.02562587099526899 0.18304152447565425
133 : None 110
conf:  None 0.32043749575602753
134 : 76 64
conf:  0.4720994028390446 0.14700837814657383
136 : None 133
conf:  None 0.1142504102580618
138 : 106 131
conf:  0.4602121839039434 0.14859702931744703
140 : 31 74
conf:  0.4584115341724631 0.1644526550717106
142 : 87 None
conf:  0.5025743023815904 None
145 : 3 None
conf:  0.5017519312086033 Non

# Scratch: sanity checks for the frames

In [ ]:
fname = r"C:\dlc_stacks\Test_project\2-training_data\all_tracklets.h5"
import pandas as pd

df = pd.read_hdf(fname)

In [ ]:
df

In [ ]:
df.diff()[1:]


In [ ]:
all_neurons = list(df.columns.levels[0])
for n in all_neurons:
    zd = df.diff()[1:][n]['z']
    if all(abs(zd) > 2):
        print(n, list(zd))

In [ ]:
np.quantile(np.arange(10), 0.9)

In [79]:
np.arange(5,6)

array([5])